<a href="https://colab.research.google.com/github/halenmona2022/regression/blob/main/notebookba913ccef8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## fairseq 模块

fairseq模块是一个基于PyTorch的开源序列建模工具库，用于研究和开发自然语言处理任务中的序列建模算法。fairseq库提供了一些常用的序列建模模型和训练/推理工具，例如transformer、LSTM、单向/双向RNN等模型，以及beam search、sampling等推理方法。fairseq还提供了高效的分布式训练模块，支持多GPU集群和多机器分布式训练，能够帮助科研人员在大规模数据上进行高质量的深度学习研究

fairseq模块包括以下几种组件：

* 数据预处理：提供数据处理的API，帮助用户从原始语言数据构建词典、分割/编码输入序列等；
* 训练管道：支持单机训练、多GPU训练和多机器分布式训练。训练过程中自动进行梯度裁剪、参数更新、优化器等操作；
* 推理/评估管道：支持beam search、sampling等推理方法，在测试集上评估最终的模型效果；
* 模型和组件：提供若干常见的序列建模模型，例如transformer、LSTM、单向/双向RNN等，同时也支持自定义模型和组件的开发；
* 优化器：实现了常用的优化器，例如Adam、Adagrad、RMSprop等


In [ ]:
# 安装fairseq
!pip install 'torch>=1.6.0' editdistance matplotlib sacrebleu sacremoses sentencepiece tqdm wandb
!pip install --upgrade jupyter ipywidgets
!git clone https://github.com/pytorch/fairseq.git
!cd fairseq && git checkout 9a1c497
!pip install --upgrade ./fairseq/

In [ ]:
#----- Environment 环境准备-----



# import python
import sys
import pdb
import pprint
import logging
import os
import random

# import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import numpy as np


# import visualization
import tqdm.auto as tqdm
from pathlib import Path
from argparse import Namespace
import matplotlib.pyplot as plt
!pip install fairseq
from fairseq import utils

# Cuda environment
cuda_env = utils.CudaEnvironment()
utils.CudaEnvironment.pretty_print_cuda_env_list([cuda_env])
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# set random seed
seed = 114514
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
# 下载资料集
data_dir = '/kaggle/working/DATA/rawdata'
dataset_name = 'ted2020'
urls = (
    "https://github.com/figisiwirf/ml2023-hw5-dataset/releases/download/v1.0.1/ml2023.hw5.data.tgz",
    "https://github.com/figisiwirf/ml2023-hw5-dataset/releases/download/v1.0.1/ml2023.hw5.test.tgz"
)
file_names = (
    'ted2020.tgz', # train & dev
    'test.tgz', # test
)
prefix = Path(data_dir).absolute() / dataset_name

prefix.mkdir(parents=True, exist_ok=True)
for u, f in zip(urls, file_names):
    path = prefix/f
    if not path.exists():
        !wget {u} -O {path}
    if path.suffix == ".tgz":
        !tar -xvf {path} -C {prefix}
    elif path.suffix == ".zip":
        !unzip -o {path} -d {prefix}
!mv {prefix/'raw.en'} {prefix/'train_dev.raw.en'}
!mv {prefix/'raw.zh'} {prefix/'train_dev.raw.zh'}
!mv {prefix/'test.en'} {prefix/'test.raw.en'}
!mv {prefix/'test.zh'} {prefix/'test.raw.zh'}

# 设置语言
src_lang = 'en'
tgt_lang = 'zh'

data_prefix = f'{prefix}/train_dev.raw'
test_prefix = f'{prefix}/test.raw'

!head {data_prefix+'.'+src_lang} -n 5
!head {data_prefix+'.'+tgt_lang} -n 5

In [ ]:
# 档案前处理
import re

# strQ2B: 将一个字符串中的全角字符转成半角字符，用于处理中文字符串
def strQ2B(ustring):
    """全角字符 -> 半角ASCII"""
    # reference:https://ithelp.ithome.com.tw/articles/10233122
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全宽空格直接换成空格
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全宽的别的字符直接转了
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)
#清洗和整理一个句子，包括去除括号中的文本、去除特殊字符、将全角字符转成半角字符、在标点符号前后添加空格
def clean_s(s, lang):
    if lang == 'en':
        s = re.sub(r"", "", s) # remove ([text]) 去掉括号内的语句
        s = s.replace('-', '') # remove '-'
        s = re.sub('([.,;!?()\"])', r' \1 ', s) # keep punctuation
    elif lang == 'zh':
        s = strQ2B(s) # Q2B
        s = re.sub(r"", "", s) # remove ([text])
        s = s.replace(' ', '')
        s = s.replace('—', '')
        s = s.replace('“', '"')
        s = s.replace('”', '"')
        s = s.replace('_', '')
        s = re.sub('([。,;!?()\"~「」])', r' \1 ', s) # keep punctuation
    s = ' '.join(s.strip().split())
    return s

#求句子长度，可以用来限制句子最大最小查高度
def len_s(s, lang):
    if lang == 'zh':
        return len(s)
    return len(s.split())

## 处理整个语料库，按照指定的方式进行清洗和处理，并移除不符合要求的句子，保存处理后的结果到新的文件中
def clean_corpus(prefix, l1, l2, ratio=9, max_len=1000, min_len=1):
    # ratio：句子长度比例的阈值，用于过滤长度差异太大的句子，默认为9。
    # 例如，如果ratio为9，则表示源语言和目标语言长度之比不能超过9或者小于1/9，否则句子将被过滤掉。
    # max_len：句子的最大长度，用于过滤长度超过指定值的句子，默认为1000。
    # min_len：句子的最小长度，用于过滤长度小于指定值的句子，默认为1。
    if Path(f'{prefix}.clean.{l1}').exists() and Path(f'{prefix}.clean.{l2}').exists():
        print(f'{prefix}.clean.{l1} & {l2} exists. skipping clean.')
        return
    with open(f'{prefix}.{l1}', 'r') as l1_in_f:
        with open(f'{prefix}.{l2}', 'r') as l2_in_f:
            with open(f'{prefix}.clean.{l1}', 'w') as l1_out_f:
                with open(f'{prefix}.clean.{l2}', 'w') as l2_out_f:
                    for s1 in l1_in_f:
                        s1 = s1.strip()
                        s2 = l2_in_f.readline().strip()
                        s1 = clean_s(s1, l1)
                        s2 = clean_s(s2, l2)
                        s1_len = len_s(s1, l1)
                        s2_len = len_s(s2, l2)
                        if min_len > 0: # remove short sentence
                            if s1_len < min_len or s2_len < min_len:
                                continue
                        if max_len > 0: # remove long sentence
                            if s1_len > max_len or s2_len > max_len:
                                continue
                        if ratio > 0: # remove by ratio of length
                            if s1_len/s2_len > ratio or s2_len/s1_len > ratio:
                                continue
                        print(s1, file=l1_out_f)
                        print(s2, file=l2_out_f)



In [ ]:
#处理语料库
clean_corpus(data_prefix, src_lang, tgt_lang)
clean_corpus(test_prefix, src_lang, tgt_lang, ratio=-1, min_len=-1, max_len=-1)

!head {data_prefix+'.clean.'+src_lang} -n 5
!head {data_prefix+'.clean.'+tgt_lang} -n 5

In [ ]:
# 拆分训练集、验证集
valid_ratio = 0.01 # 3000~4000 would suffice
train_ratio = 1 - valid_ratio

# 首先会检查是否已经存在了划分好的训练集和验证集，如果存在则跳过这个过程
if (prefix/f'train.clean.{src_lang}').exists() \
and (prefix/f'train.clean.{tgt_lang}').exists() \
and (prefix/f'valid.clean.{src_lang}').exists() \
and (prefix/f'valid.clean.{tgt_lang}').exists():
    print(f'train/valid splits exists. skipping split.')
# 如果没有，首先读入语料库的行数，并随机打乱行的顺序
else:
    line_num = sum(1 for line in open(f'{data_prefix}.clean.{src_lang}'))
    labels = list(range(line_num))
    random.shuffle(labels)
    # 然后对于每个语言，分别打开训练集和验证集文件，并按照指定的比例将每行数据分配到训练集或验证集中去
    for lang in [src_lang, tgt_lang]:
        train_f = open(os.path.join(data_dir, dataset_name, f'train.clean.{lang}'), 'w')
        valid_f = open(os.path.join(data_dir, dataset_name, f'valid.clean.{lang}'), 'w')
        count = 0
        for line in open(f'{data_prefix}.clean.{lang}', 'r'):
            if labels[count]/line_num < train_ratio:
                train_f.write(line)
            else:
                valid_f.write(line)
            count += 1
        train_f.close()
        valid_f.close()



Subword Units

翻譯存在的一大問題是未登錄詞(out of vocabulary)，可以使用 subword units 作為斷詞單位來解決。

* 使用 [sentencepiece](#kudo-richardson-2018-sentencepiece) 套件
  
* 用 unigram 或 byte-pair encoding (BPE)
  

* 词汇表用来**将文本数据中的单词映射成机器可读的数字序列**，从而方便输入到神经网络中进行计算。
  
* 在翻译任务中，同时需要为源语言和目标语言建立两个不同的词汇表，这样可以使得源语言和目标语言的单词不发生冲突，从而避免翻译过程中的错误。
  
* 分词器则用于**将句子拆分成不同的单词或字**，使得机器可以更加细粒度地处理文本数据。
  
* 在中文等非空格分隔的语言中，分词器尤为重要，因为无法单纯地按照空格对文本进行分词。
  
* SentencePiece是一种通用的分词器，可以用于任意语言的分词和字级别建模，极大地方便了跨语言机器翻译的处理和训练。

分词器，将句子分成词语
词将词映射成机器能读的数字

In [ ]:
#Subword Units ？？？
'''
Subword Units
Out of vocabulary (OOV) has been a major problem in machine translation.
This can be alleviated by using subword units.
语汇不足(OOV)一直是机器翻译中的主要问题。这可以通过使用子词单位来缓解。

We will use the sentencepiece package
select 'unigram' or 'byte-pair encoding (BPE)' algorithm
我们将使用句子件包装
选择'unigram'或'byte-pair encoding (BPE)'算法
'''
import sentencepiece as spm
# 训练SentencePiece模型，以构建字典和分词器，用于后续的机器翻译任务。训练过程主要包括设置训练参数和调用
vocab_size = 8000
# 首先检查模型是否已经存在，如果存在则跳过训练过程
if (prefix/f'spm{vocab_size}.model').exists():
    print(f'{prefix}/spm{vocab_size}.model exists. skipping spm_train.')
# 如果模型不存在，则调用SentencePieceTrainer.train函数进行训练
else:
    spm.SentencePieceTrainer.train(
        input=','.join([f'{prefix}/train.clean.{src_lang}', # 输入文件
                        f'{prefix}/valid.clean.{src_lang}',
                        f'{prefix}/train.clean.{tgt_lang}',
                        f'{prefix}/valid.clean.{tgt_lang}']),
        model_prefix=prefix/f'spm{vocab_size}', # 输出模型前缀
        vocab_size=vocab_size, # 字典大小
        character_coverage=1, # 字符覆盖率
        model_type='unigram', # 模型类型， 'bpe' 也可
        input_sentence_size=1e6, # 输入句子大小
        shuffle_input_sentence=True, # 是否随机打乱输入句子和归一化规则
        normalization_rule_name='nmt_nfkc_cf',
    )

# 训练完成，我们就可以使用SentencePiece分词器来对输入文本进行分词，以用于后续的机器翻译任务
spm_model = spm.SentencePieceProcessor(model_file=str(prefix/f'spm{vocab_size}.model'))
# 定义字典in_tag，用于指定输入文件的不同标签
in_tag = {
    'train': 'train.clean',
    'valid': 'valid.clean',
    'test': 'test.raw.clean',
}
# 后遍历训练集、验证集和测试集，对于每个语言，分别对其进行编码。
for split in ['train', 'valid', 'test']:
    for lang in [src_lang, tgt_lang]:
        out_path = prefix/f'{split}.{lang}'
        # 如果已经存在了输出文件，则跳过编码过程
        if out_path.exists():
            print(f"{out_path} exists. skipping spm_encode.")
         # 否则，打开输入文件和输出文件，并遍历输入文件中的每一行。
        # 对于每一行，首先进行去除末尾空格的操作，然后调用SentencePiece模型的encode函数进行编码。
        # 编码后的结果以空格分隔，写入输出文件即可
        else:
            with open(prefix/f'{split}.{lang}', 'w') as out_f:
                with open(prefix/f'{in_tag[split]}.{lang}', 'r') as in_f:
                    for line in in_f:
                        line = line.strip()
                        tok = spm_model.encode(line, out_type=str)
                        print(' '.join(tok), file=out_f)

# 最后，经过编码的语料文件将被用于机器翻译模型的训练和评估




In [ ]:
!head {data_dir+'/'+dataset_name+'/train.'+src_lang} -n 5
!head {data_dir+'/'+dataset_name+'/train.'+tgt_lang} -n 5


##  用 fairseq 將資料轉為 binary
将单词变成数字

In [ ]:
#Binarize the data with fairseq
'''
Prepare the files in pairs for both the source and target languages.
\ In case a pair is unavailable, generate a pseudo pair to facilitate binarization.
为源语言和目标语言准备成对的文件。
如果一对不可用，则生成一个伪对以方便二值化。
binpath = Path('./DATA/data-bin', dataset_name)
'''
# 使用fairseq_cli对上述已编码的语料库进行预处理，以生成可用于机器翻译训练的数据集`
binpath = Path('./DATA/data-bin', dataset_name)
# 使用fairseq_cli对上述已编码的语料库进行预处理，以生成可用于机器翻译训练的数据集
if binpath.exists():
    print(binpath, "exists, will not overwrite!")
# 调用了fairseq_cli.preprocess模块将已编码的语料库转换为二进制格式的数据集，以供后续的机器翻译模型训练和评估
else:
    # --source-lang和--target-lang参数，指定了源语言和目标语言的缩写
    # --trainpref、--validpref和--testpref指定了相应语料库的前缀，也就是数据集的输入文件。这些文件应该是已按照标记进行编码的文件
    # --destdir指定了输出数据集文件夹路径
    # --joined-dictionary参数表示共享源语言和目标语言的字典，以减小数据集大小并减小训练时间
    # --workers参数指定了使用的进程数。在预处理语料库时，可以使用多个进程并行处理以提高速度
    !python -m fairseq_cli.preprocess \
        --source-lang {src_lang}\
        --target-lang {tgt_lang}\
        --trainpref {prefix/'train'}\
        --validpref {prefix/'valid'}\
        --testpref {prefix/'test'}\
        --destdir {binpath}\
        --joined-dictionary\
        --workers 2

超参数的设定

In [ ]:
# ----- Hyper parameter 超参数设定-----
config = Namespace(
    datadir = "/kaggle/working/DATA/data-bin/ted2020",
    savedir = "/kaggle/working/rnn",
    source_lang = src_lang,
    target_lang = tgt_lang,

    # cpu threads when fetching & processing data.
    # 指定了在数据获取和处理时使用的CPU线程数
    num_workers=0,
    # batch size in terms of tokens.
    #  max_tokens指定了每个训练批次的最大令牌数量，
    max_tokens=8192,
    # gradient accumulation increases the effective batchsize.
    # 即在每次反向传播时考虑多少个标记，为了提高训练效率，通常使用梯度积累
    # 训练两次把梯度累计起来进行一次更新
    accum_steps=2,

    # the lr s calculated from Noam lr scheduler. 可以修改 the maximum lr by this factor.
    # 动态学习率
    lr_factor=2.,
    lr_warmup=4000,

    # clipping gradient norm helps alleviate gradient exploding
    #梯度裁剪的预值,以便减轻梯度爆炸的问题
    clip_norm=1.0,

    # maximum epochs for training
    max_epoch=16,
    start_epoch=1,

    # beam size for beam search
    # 波束搜索的大小>
    beam=5,
    # 最长输出长度为 ax + b,
    # generate sequences of maximum length ax + b, where x is the source length
    # max_len_a和max_len_b指定了在解码时生成的最大输出长度。这允许在单个模型中支持可变长度的输出
    max_len_a=1.2,
    max_len_b=10,
    # when decoding, post process sentence by removing sentencepiece symbols.
    # 指定了解码后的后处理方法，以去除SentencePiece等标记
    post_process = "sentencepiece",

    # checkpoints
    # keep_last_epochs指定了应保留的最后几个训练轮数据。
    # 这些数据将用于用新数据替换它们以进行重新训练。这有助于避免过拟合和错误方向的训练，反复使用相同数据构造模型
    keep_last_epochs=5,
    resume=None, # 是否从之前的检查点训练模型

    # logging
    use_wandb=False,
)



1. 梯度积累：
  
  * 概念：指将多个小批次的梯度累加起来，然后一起执行一次反向传播，并更新模型的参数
  * 用途：这样可以将小批次的效率优劣通过累加来平衡，提升梯度的准确性和稳定性
  * 累积的次数：（即`accum_steps`）越多，每次反向传播计算的梯度越准确，但训练时间也会减慢
2. `beam search` 束搜索 束搜索是一种常用的**生成式建模方法**。束搜索的基本思想是在*输入源文本的基础*上，通过一个经过训练的神经网络模型来生成目标文本。生成时通过保留预测概率最高的K个结果，同时基于每个结果继续扩大搜索范围，以提高翻译准确率。这个搜索过程中，保留预测概率最高的K个结果称为“束”
  
  * 增加`beam size`有助于提高翻译的准确性，但会显著降低翻译过程的速度和效率
3. `resume`
  
  * `resume`参数用于指定是否从之前的检查点继续训练模型。在本例中，resume的值为None，表示不从检查点继续训练。如果将resume设置为之前的检查点文件名，那么训练将从上一次保存的检查点处继续训练
  * 由于某些原因（如服务器故障、网络中断等），训练被中断，需要在此基础上继续进行训练
  * 对于非常复杂的模型，模型的训练过程需要数天、数周，为了避免从头开始重新训练，通常会将训练拆分为多个阶段，并在每个阶段结束时保存检查点。在这种情况下，如果要对同一模型进行另一项任务的训练，可以在之前的检查点基础上继续训练，从而节省训练时间和计算资源
4. `use_wandb`
  
  * use_wandb参数用于指定是否使用W&B库（即Weights & Biases库）进行实时记录和可视化训练过程和指标
  * W&B是一个工具，可以记录和可视化机器学习模型训练过程中的各种指标和信息，帮助用户更好地理解模型的性能和训练过程。使用W&B库可以实时查看模型的训练/验证/测试曲线、参数的分布情况等

In [ ]:
#Logging套件
logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level="INFO", # "DEBUG" "WARNING" "ERROR"
    stream=sys.stdout,
)
proj = "hw5.seq2seq"
logger = logging.getLogger(proj)
if config.use_wandb:
    import wandb
    wandb.init(project=proj, name=Path(config.savedir).stem, config=config)

讀取資料集

## 借用 fairseq 的 TranslationTask

* 用來讀進上面 binarized 的檔案
* 有現成的 data iterator (dataloader)
* 字典 task.source_dictionary 和 task.target_dictionary 也很好用
* 有實做 beam search

In [ ]:
# ------ Dataset & Dataloader ------

#Dataloading

# 从fairseq设置一个机器翻译任务，返回一个translationTask操作对象
from fairseq.tasks.translation import TranslationConfig, TranslationTask

# 配置机器翻译任务的参数
task_cfg = TranslationConfig(
    data=config.datadir, #数据路径
    source_lang=config.source_lang,# 源语言
    target_lang=config.target_lang,# 目标语言
    train_subset="train", # 训练集子集
    required_seq_len_multiple=8,
    dataset_impl="mmap",
    upsample_primary=1,
)
# 用于创建一个TranslationTask操作对象，
# 其内部会根据上一步的参数配置来构建相应的模型和数据处理流程
task = TranslationTask.setup_task(task_cfg)


In [ ]:

logger.info("loading data for epoch 1")
# 调用TranslationTask对象的load_dataset()方法，从指定数据路径中读取处于训练集中的数据，并经过预处理后存储于缓存中
# split="train"表示加载训练集，epoch=1表示读取处于第1个epoch时的训练数据，combine=True表示如果有回译数据可用，则将其与训练数据合并再进行进一步处理
task.load_dataset(split="train", epoch=1, combine=True) # combine if you have back-translation data.
task.load_dataset(split="valid", epoch=1)


In [ ]:
# 从fairseq任务的验证数据集中随机采样一个样本，并输出其源语言和目标语言文本内容
# 展示fairseq任务数据集中样本的内容，以检验数据的正确性和准确性`
sample = task.dataset("valid")[1] # 从fairseq的Validation数据集中获取索引为1的样本
pprint.pprint(sample)
# 使用fairseq提供的string()函数将该样本的源语言内容转换为文本形式，并输出
pprint.pprint(
    "Source: " + \
    task.source_dictionary.string(
        sample['source'],
        config.post_process,
    )
)
# 将目标语言转化为文本并输出
pprint.pprint(
    "Target: " + \
    task.target_dictionary.string(
        sample['target'],
        config.post_process,
    )
)


Dataset Iterator

* 將每個 batch 控制在 N 個 token 讓 GPU 記憶體更有效被利用
  
* 讓 training set 每個 epoch 有不同 shuffling
  
* 濾掉長度太長的句子
  
* 將每個 batch 內的句子 pad 成一樣長，好讓 GPU 平行運算
  
* 加上 eos 並 shift 一格
  
  * teacher forcing: 為了訓練模型根據prefix生成下個字，decoder的輸入會是輸出目標序列往右shift一格。
    
  * 一般是會在輸入開頭加個bos token (如下圖)”![seq2seq](https://i.imgur.com/0zeDyuI.png)
    
  * fairseq 則是直接把 eos 挪到 beginning，訓練起來效果其實差不多。例如:
    
        # 輸出目標 (target) 和 Decoder輸入 (prev_output_tokens):
                     eos = 2
                  target = 419,  711,  238,  888,  792,   60,  968,    8,    2
        prev_output_tokens = 2,  419,  711,  238,  888,  792,   60,  968,    8

In [ ]:
# 根据fairseq任务和数据集配置参数，创建一个数据迭代器，并使用该迭代器按batch逐一获取训练或验证数据
# 主要目的是为了检查数据是否被正确读入，并且查看当前batch的训练数据结构是否满足需求

def load_data_iterator(task, split, epoch=1, max_tokens=4000, num_workers=1, cached=True):
    batch_iterator = task.get_batch_iterator(
        dataset=task.dataset(split),
        max_tokens=max_tokens, #一个batch中最多多少个token
        max_sentences=None,
        max_positions=utils.resolve_max_positions( # 确保可以处理的序列最大长度（以 tokens 为单位）
            task.max_positions(),
            max_tokens,
        ),
        ignore_invalid_inputs=True, # 跳过无效的输入数据(数据格式不正确，长度为0，无效字符之类的)
        seed=seed,
        num_workers=num_workers,
        epoch=epoch,
        disable_iterator_cache=not cached,
        # Set this to False to speed up. However, if set to False, changing max_tokens beyond
        # first call of this method has no effect.
    )
    return batch_iterator

demo_epoch_obj = load_data_iterator(task, "valid", epoch=1, max_tokens=20, num_workers=1, cached=False)
demo_iter = demo_epoch_obj.next_epoch_itr(shuffle=True) # 获取下一个batch的样本
sample = next(demo_iter)#获取迭代器中下一个元素，即本次迭代的数据样本。得到的sample对象包括了当前batch内所有样本的信息
sample

函数 load_data_iterator 的参数说明：

* `task`: 模型任务。
* `split`: 读取哪部分的数据集，可以是 "train" 或 "valid"。
* `epoch`: 当前轮次的训练迭代次数，默认为 1。
* `max_tokens`: 每个 batch 中最多包含多少个 token，用于控制内存使用和训练速度。
* `num_workers`: 用于数据加载的线程数。
* `cached`: 是否缓存迭代器。

返回值 `batch_iterator` 是通过 `get_batch_iterator` 方法创建的迭代器，其中包括数据集、batch大小等参数。返回的迭代器对象是 `FairseqIterativeDataset` 类的对象，可以用于获取和处理数据集中的数据。next_epoch_itr(shuffle=True) 方法用于获取下一个 batch 的样本，可以通过 next 函数获取迭代器中下一个元素，即本次迭代的数据样本，这个样本对象包括了当前 batch 内所有样本的信息。

* 每個 batch 是一個字典，key 是字串，value 是 Tensor，內容說明如下
```python
batch = {
  "id": id, # 每個 example 的 id
  "nsentences": len(samples), # batch size 句子數
  "ntokens": ntokens, # batch size 字數
  "net_input": {
      "src_tokens": src_tokens, # 來源語言的序列
      "src_lengths": src_lengths, # 每句話沒有 pad 過的長度
      "prev_output_tokens": prev_output_tokens, # 上面提到右 shift 一格後的目標序列
  },
  "target": target, # 目標序列
}
```

定義模型架構

* 我們一樣繼承 fairseq 的 encoder, decoder 和 model, 這樣測試階段才能直接用他寫好的 beam search 函式

In [ ]:
from fairseq.models import (
    FairseqEncoder, # fairseq中的Encoder基类，所有Encoder模块都必须继承该类
    FairseqIncrementalDecoder, # fairseq中的Incremental Decoder基类，所有Incremental Decoder模块都必须继承该类
    FairseqEncoderDecoderModel # Encoder-Decoder模型的基类，是实现sequence-to-sequence任务的核心
)

Encoder 編碼器

* seq2seq 模型的編碼器為 RNN 或 Transformer Encoder，以下說明以 RNN 為例，Transformer 略有不同。對於每個輸入，Encoder 會輸出一個向量和一個隱藏狀態(hidden state)，並將隱藏狀態用於下一個輸入。換句話說，Encoder 會逐步讀取輸入序列，並在每個 timestep 輸出單個向量，以及在最後 timestep 輸出最終隱藏狀態(content vector)
  
* 參數:
  
  * *args*
    * encoder_embed_dim 是 embedding 的維度，主要將 one-hot vector 的單詞向量壓縮到指定的維度，主要是為了降維和濃縮資訊的功用
    * encoder_ffn_embed_dim 是 RNN 輸出和隱藏狀態的維度(hidden dimension)
    * encoder_layers 是 RNN 要疊多少層
    * dropout 是決定有多少的機率會將某個節點變為 0，主要是為了防止 overfitting ，一般來說是在訓練時使用，測試時則不使用
  * *dictionary*: fairseq 幫我們做好的 dictionary. 在此用來得到 padding index，好用來得到 encoder padding mask.
  * *embed_tokens*: 事先做好的詞嵌入 (nn.Embedding)
* 輸入:
  
  * *src_tokens*: 英文的整數序列 e.g. 1, 28, 29, 205, 2
* 輸出:
  
  * *outputs*: 最上層 RNN 每個 timestep 的輸出，後續可以用 Attention 再進行處理
  * *final_hiddens*: 每層最終 timestep 的隱藏狀態，將傳遞到 Decoder 進行解碼
  * *encoder_padding_mask*: 告訴我們哪些是位置的資訊不重要。

### timestep（时间步）

* 概述：用于描述一个序列模型处理序列数据的过程，其中每个时间步处理序列的一个元素
* 在seq2seq模型的编码器中，timestep指的是编码器在处理输入序列时所处的时刻数，也即是处理序列中的第几个元素
* 编码器会对输入序列进行逐个元素处理，每处理一个元素就会输出一个向量和一个隐状态，并将隐状态用于下一个输入元素的处理。
* 在每个timestep上，编码器都会处理输入序列的一个元素，一直处理到序列的末尾，最后输出一个最终的隐状态或者是context vector

In [ ]:

#Rnn？？
class RNNEncoder(FairseqEncoder): #注意这逼fairseq
    def __init__(self, args, dictionary, embed_tokens):
        super().__init__(dictionary)
        #继承args罢
        self.embed_tokens = embed_tokens
        self.embed_dim = args.encoder_embed_dim
        self.hidden_dim = args.encoder_ffn_embed_dim # 每个GRU单元的隐藏层维度
        self.num_layers = args.encoder_layers # num_layers是GRU的层数
         # 创建一个dropout_in_module实例
        self.dropout_in_module = nn.Dropout(args.dropout)
        #看下面(恼)
        self.rnn = nn.GRU(
            self.embed_dim,
            self.hidden_dim,
            self.num_layers,
            dropout=args.dropout,
            # 设置batch_first=False（即不使用batch_first模式）
            # bidirectional=True，表示该GRU是双向的
            batch_first=False,
            bidirectional=True
        )
        self.dropout_out_module = nn.Dropout(args.dropout)

        self.padding_idx = dictionary.pad() # 填充字符的索引

    # 将双向GRU的hidden state沿着最后一维（即direction维）拼接在一起
    def combine_bidir(self, outs, bsz: int):
        out = outs.view(self.num_layers, 2, bsz, -1).transpose(1, 2).contiguous()
        return out.view(self.num_layers, bsz, -1)

    def forward(self, src_tokens, **unused): # src_tokens是待编码的源序列输入
        bsz, seqlen = src_tokens.size() # 获取bsz和seqlen分别为输入src_tokens的batch size和序列长度
        # embedding层
         # 输入的src_tokens进行embedding并赋值给x
        x = self.embed_tokens(src_tokens)
        x = self.dropout_in_module(x)
        # B x T x C -> T x B x C 以适应双向GRU的输入格式
        x = x.transpose(0, 1)
        # 双向RNN
        h0 = x.new_zeros(2 * self.num_layers, bsz, self.hidden_dim)
        x, final_hiddens = self.rnn(x, h0)
        outputs = self.dropout_out_module(x)
        # outputs = [sequence len, batch size, hid dim * directions]
        # hidden =  [num_layers * directions, batch size  , hid dim]

        # 由于编码器是双向的，我们需要将两个方向的隐藏状态连接起来
        final_hiddens = self.combine_bidir(final_hiddens, bsz)
        # hidden =  [num_layers x batch x num_directions*hidden]

        #这啥啊
        encoder_padding_mask = src_tokens.eq(self.padding_idx).t()
        return tuple(
            (
                outputs,  # seq_len x batch x hidden outputs为全序列的每个位置上模型输出的最后一层RNN隐状态
                final_hiddens,  # num_layers x batch x num_directions*hidden final_hiddens为最终的隐藏状态
                encoder_padding_mask,  # seq_len x batch 进行padding的输入标记，形状为(T, B)。
            )
        )
    #不需要理这些事（恼）
    def reorder_encoder_out(self, encoder_out, new_order):
         # 這個beam search時會用到，意義並不是很重要
        # new_order是一个张量，代表新的排列顺序。这里我们假设new_order的形状为(batch_size, )
        # 根据给定的新顺序new_order，重新排列encoder输出
        return tuple(
            (
                encoder_out[0].index_select(1, new_order),
                encoder_out[1].index_select(1, new_order),
                encoder_out[2].index_select(1, new_order),
            )
        )


Attention

* 當輸入過長，或是單獨靠 “content vector” 無法取得整個輸入的意思時，用 Attention Mechanism 來提供 Decoder 更多的資訊
  
* 根據現在 **Decoder embeddings** ，去計算在 **Encoder outputs** 中，那些与其有较高的关系，根据关系的數值來把 Encoder outputs 平均起來作為 **Decoder** RNN 的輸入
  
* 常見 Attention 的實作是用 Neural Network / Dot Product 來算 **query** (decoder embeddings) 和 **key** (Encoder outputs) 之間的關係，再對所有算出來的數值做 **softmax** 得到分布，最後根据这个分布對 **values** (Encoder outputs) 做 **weight sum**
  
* 參數:
  
  * *input_embed_dim*: key 的维度，应该是 decoder 要做 attend 時的向量的维度
  * *source_embed_dim*: query 的维度，應是要被 attend 的向量(encoder outputs)的维度
  * *output_embed_dim*: value 的维度，應是做完 attention 後，下一層預期的向量维度
* 輸入:
  
  * *inputs*: 就是 key，要 attend 別人的向量
  * *encoder_outputs*: 是 query/value，被 attend 的向量
  * *encoder_padding_mask*: 告訴我們哪些是位置的資訊不重要。
* 輸出:
  
  * *output*: 做完 attention 后的 context vector（上下文向量），来表示解码器对于编码器输出序列的"注意力"。最后，Context Vector将会被拼接到Decoder的输入中，以帮助Decoder生成下一个输出词语。
  * *attention score*: attention 的分布

### Attention机制

1. `Key Vector`：Key Vector是需要被attend的向量，通常是**编码器（Encoder）的输出序列**（也可以是其他任何与当前任务相关的向量）
2. `Query Vector`：Query Vector通常是**解码器（Decoder）的当前隐藏状态**（hidden state），即Decoder的当前隐层状态
3. `Value Vector`：Value Vector是Key Vector的值，**即在Encoder输出序列中的每个词语的表示向量**。在Seq2Seq模型中，这个向量通常就是Encoder输出的状态。

* 在Seq2Seq模型中，Encoder的输出对应Value，而Decoder的隐藏状态对应Query，我们需要计算出Decoder向量与每个Encoder输出之间的关系，并根据这个关系进行加权求和。
* `Query Vector`通常与`Key Vector`之间的相似度（或关系）用于计算权重：
  * attention_score = softmax(score)
  * context_vector = sum(attention_score * Value)
* 其中，`attention_score`为一个权重向量，它的和为1，这样我们就得到了加权求和的上下文向量context_vector。在Seq2Seq模型中，这个上下文向量将与Decoder的当前状态进行拼接，作为Decoder的下一个输入
* Mask：需要注意的是，在计算Attention Score之前，还需要应用一个掩码（mask）来过滤掉Padding的位置，从而得到正确的attention分布。在Seq2Seq模型中，这个掩码将由Encoder的padding mask传递给Attention。(只看左边)

In [ ]:
#Attention
class AttentionLayer(nn.Module):
    def __init__(self, input_embed_dim, source_embed_dim, output_embed_dim, bias=False):
        super().__init__()
        #输入输出层先定义好
        self.input_proj = nn.Linear(input_embed_dim, source_embed_dim, bias=bias)
        self.output_proj = nn.Linear(
            input_embed_dim + source_embed_dim, output_embed_dim, bias=bias
        )

    def forward(self, inputs, encoder_outputs, encoder_padding_mask):
        # inputs: T, B, dim
        # encoder_outputs: S x B x dim
        # padding mask:  S x B

        # 先把维度调整好
        inputs = inputs.transpose(1,0) # B, T, dim
        encoder_outputs = encoder_outputs.transpose(1,0) # B, S, dim
        encoder_padding_mask = encoder_padding_mask.transpose(1,0) # B, S

        #输入层，先过liner调整维数
        x = self.input_proj(inputs)

        # compute attention
        # (B, T, dim) x (B, dim, S) = (B, T, S)
        attn_scores = torch.bmm(x, encoder_outputs.transpose(1,2))

        # # 擋住padding位置的attention
        if encoder_padding_mask is not None:
             # 利用broadcast  B, S -> (B, 1, S)
            encoder_padding_mask = encoder_padding_mask.unsqueeze(1) #B, S -> (B, 1, S)
            attn_scores = (
                attn_scores.float()
                .masked_fill_(encoder_padding_mask, float("-inf"))
                .type_as(attn_scores)
            )  # FP16 support: cast to float and back ？？

        # 给对应原序列的层来个softmax
        attn_scores = F.softmax(attn_scores, dim=-1)

        # shape (B, T, S) x (B, S, dim) = (B, T, dim) weighted sum
        # 批量矩阵乘法，即将两个三维张量中的每个矩阵相乘，得到一个新的三维张量
        x = torch.bmm(attn_scores, encoder_outputs)

        # (B, T, dim)
        x = torch.cat((x, inputs), dim=-1) #拼接
        x = torch.tanh(self.output_proj(x)) # concat + linear + tanh

        # restore shape (B, T, dim) -> (T, B, dim)
        return x.transpose(1,0), attn_scores




Decoder 解碼器

* 解码器的 `hidden states` 會用编码器最终隐藏状态來初始化(`content vector`)
* 解码器同時也根据目前 `timestep` 的輸入(也就是前几个 timestep 的 output)，改变 `hidden states`，並輸出結果
* 如果加入 `attention` 可以使表現更好
* 我們把 seq2seq 步驟寫在解码器里，好讓等等 Seq2Seq 這個型別可以通用 RNN 和 Transformer，而不用再改寫

* 参数:
  * *args*
    * `decoder_embed_dim` 是解码器 embedding 的維度，类同 encoder_embed_dim，
    * `decoder_ffn_embed_dim` 是解码器 RNN 的隐藏维度，類同 encoder_ffn_embed_dim
    * `decoder_layers` 解码器 RNN 的层数
    * `share_decoder_input_output_embed`通常 decoder 最後輸出的投影矩陣會和輸入 embedding 共用參數
  * *dictionary*: fairseq 幫我們做好的 dictionary.
  * *embed_tokens*: 事先做好的词嵌入(nn.Embedding)
* 輸入:
  * *prev_output_tokens*: 英文的整數序列 e.g. 1, 28, 29, 205, 2 已經 shift 一格的 target
  * *encoder_out*: 編碼器的輸出
  * *incremental_state*: 這是測試階段為了加速，所以會記錄每個 timestep 的 hidden state 詳見 forward
* 輸出:
  * *outputs*: decoder 每個 timestep 的 logits，還沒經過 softmax 的分布
  * *extra*: 沒用到

In [ ]:

#Decoder 酷诶，但是换成transformer了捏
#但是学学（悲）
class RNNDecoder(FairseqIncrementalDecoder):
    def __init__(self, args, dictionary, embed_tokens): #arg是一个参数
        super().__init__(dictionary)
        self.embed_tokens = embed_tokens

        #对args进行一个继承！！
        assert args.decoder_layers == args.encoder_layers, f"""seq2seq rnn requires that encoder
        and decoder have same layers of rnn. got: {args.encoder_layers, args.decoder_layers}"""
        assert args.decoder_ffn_embed_dim == args.encoder_ffn_embed_dim*2, f"""seq2seq-rnn requires
        that decoder hidden to be 2*encoder hidden dim. got: {args.decoder_ffn_embed_dim, args.encoder_ffn_embed_dim*2}"""

        self.embed_dim = args.decoder_embed_dim
        self.hidden_dim = args.decoder_ffn_embed_dim
        self.num_layers = args.decoder_layers
        self.dropout_in_module = nn.Dropout(args.dropout) #抛dropout还要单独标记嘛

        self.rnn = nn.GRU( #多层门控循环单元循环神经网络
            self.embed_dim, #输入特征数
            self.hidden_dim, #隐藏状态特征数量
            self.num_layers, #层数
            dropout=args.dropout, #丢弃率
            batch_first=False, #批次优先no
            bidirectional=False #双向NO
        )
        self.attention = AttentionLayer( #Attention没啥好说
            self.embed_dim, self.hidden_dim, self.embed_dim, bias=False
        )
        self.dropout_out_module = nn.Dropout(args.dropout)

        #如果hidden_dim!=embed_dim，那么加一层linear变化
        if self.hidden_dim != self.embed_dim:
            self.project_out_dim = nn.Linear(self.hidden_dim, self.embed_dim)
        else:
            self.project_out_dim = None

        #如果输入输出embed层相同，那么输出project层直接用相同的embed
        if args.share_decoder_input_output_embed:
            self.output_projection = nn.Linear(
                self.embed_tokens.weight.shape[1],
                self.embed_tokens.weight.shape[0],
                bias=False,
            )
            self.output_projection.weight = self.embed_tokens.weight
        #否则自己开一个dict的embed
        else:
            self.output_projection = nn.Linear(
                self.output_embed_dim, len(dictionary), bias=False
            )
            nn.init.normal_( #正态分布中生成值填充该层参数，实现神经网络参数初始化
                self.output_projection.weight, mean=0, std=self.output_embed_dim ** -0.5
            )

    def forward(self, prev_output_tokens, encoder_out, incremental_state=None, **unused):
        #  从encoder中弄出output
        encoder_outputs, encoder_hiddens, encoder_padding_mask = encoder_out
        # outputs:          seq_len x batch x num_directions*hidden
        # encoder_hiddens:  num_layers x batch x num_directions*encoder_hidden
        # padding_mask:     seq_len x batch

        if incremental_state is not None and len(incremental_state) > 0:
            # 如果上一个时间步的信息被保留，我们可以从那里继续，而不是从bos开始
            prev_output_tokens = prev_output_tokens[:, -1:]
            cache_state = self.get_incremental_state(incremental_state, "cached_state")
            prev_hiddens = cache_state["prev_hiddens"]
        else:
            # 增量状态不存在，要么是训练时间，要么是测试时间的第一个时间步
            # 为seq2seq做准备:将encoder_hidden传递给解码器隐藏状态
            prev_hiddens = encoder_hiddens

        bsz, seqlen = prev_output_tokens.size()

        # embed tokens
        x = self.embed_tokens(prev_output_tokens)
        x = self.dropout_in_module(x)
        x = x.transpose(0, 1) # B x T x C -> T x B x C

        # decoder-to-encoder attention
        if self.attention is not None:
            x, attn = self.attention(x, encoder_outputs, encoder_padding_mask)

        # 通过单向RNN
        x, final_hiddens = self.rnn(x, prev_hiddens) #多层门控循环单元循环神经网络
        # outputs = [sequence len, batch size, hid dim]
        # hidden =  [num_layers * directions, batch size  , hid dim]
        x = self.dropout_out_module(x)

        # 项目的嵌入大小(如果hidden与嵌入大小不同，且share_embedding为True，那我们需要额外来一层线性层
        if self.project_out_dim != None:
            x = self.project_out_dim(x)

        # project to vocab size
        # # 投影到vocab size 的分佈
        x = self.output_projection(x)

        # T x B x C -> B x T x C
        x = x.transpose(1, 0)

        # 如果是增量，则记录当前时间步的隐藏状态，这些状态将在下一个时间步恢复？
        #  # 如果是Incremental, 記錄這個timestep的hidden states, 下個timestep讀回來
        cache_state = {
            "prev_hiddens": final_hiddens,
        }
        self.set_incremental_state(incremental_state, "cached_state", cache_state)

        return x, None

    #这是啥似乎不用理
    def reorder_incremental_state(
        self,
        incremental_state,
        new_order,
    ):
        # This is used by fairseq's beam search. How and why is not particularly important here.
        cache_state = self.get_incremental_state(incremental_state, "cached_state")
        prev_hiddens = cache_state["prev_hiddens"]
        prev_hiddens = [p.index_select(0, new_order) for p in prev_hiddens]
        cache_state = {
            "prev_hiddens": torch.stack(prev_hiddens),
        }
        self.set_incremental_state(incremental_state, "cached_state", cache_state)
        return




Seq2Seq

* 由 **Encoder** 和 **Decoder** 組成
* 接收輸入並傳給 **Encoder**
* 將 **Encoder** 的輸出傳給 **Decoder**
* **Decoder** 根據前幾個 timestep 的輸出和 **Encoder** 輸出進行解碼
* 當解碼完成後，將 **Decoder** 的輸出傳回

### 用 RNN 实现和 用 Transformer 实现的区别

1. `RNN`
  
  * 编码器和解码器通常都采用循环神经网络结构，如LSTM或GRU。
  * RNN能够较好地处理序列中的时序信息，但容易在处理长序列时出现梯度消失或爆炸等问题。
  * 因此，为了解决这些问题，通常需要通过采用注意力机制（Attention）或卷积神经网络（CNN）等手段来提高模型的性能。
2. `Transformer`
  
  * 不需要采用循环神经网络，而是采用了全新的自注意力机制（Self-Attention）和多头注意力机制（Multi-Head Attention）,从而避免了RNN中的梯度消失和爆炸问题
    
  * Transformer的自注意力机制允许模型在不考虑时序的情况下处理序列数据，而多头注意力机制可以理解为同时进行多个注意力的训练，进一步提高了模型的表现力和复杂度。
    
    * 具体而言，Transformer模型包括**编码器和解码器**两个部分，每个部分由多个完全相同的模块组成。模块内部包含子层，其中包含自注意力机制和前向神经网络（Feed-Forward Neural Network），模块之间使用多头注意力机制进行连接。这种设计使得Transformer能够高效地处理序列数据，特别是长序列数据。

总的来说，使用Transformer实现Seq2Seq模型相对于使用RNN实现Seq2Seq模型，具有训练速度更快、处理长序列时出现的梯度消失和爆炸问题更少、表现更优秀等优势。

In [ ]:
#Seq2Seq 把encoder和decoder接一起
class Seq2Seq(FairseqEncoderDecoderModel): # 继承了基类FairseqEncoderDecoderModel
    def __init__(self, args, encoder, decoder):
        super().__init__(encoder, decoder)
        self.args = args

    def forward(
        self,
        src_tokens, #原tocken
        src_lengths,
        prev_output_tokens, #准备输入decoder的token?
        return_all_hiddens: bool = True,
    ):
        #就是把encoder和decoder接起来，没了
        encoder_out = self.encoder(
            src_tokens, src_lengths=src_lengths, return_all_hiddens=return_all_hiddens
        )
        logits, extra = self.decoder(
            prev_output_tokens, #输入token
            encoder_out=encoder_out, #把encoder输出输入decoder
            src_lengths=src_lengths, #长度
            return_all_hiddens=return_all_hiddens, # True就会extra吗
        )
        return logits, extra # 预测的结果logits和一些附加信息extra

In [ ]:
#模型实例化

# # HINT: transformer architecture
from fairseq.models.transformer import (
    TransformerEncoder,
    TransformerDecoder,
)

def build_model(args, task):
    """ 按照參數設定建置模型 """
    src_dict, tgt_dict = task.source_dictionary, task.target_dictionary

    # 詞嵌入
    encoder_embed_tokens = nn.Embedding(len(src_dict), args.encoder_embed_dim, src_dict.pad())
    decoder_embed_tokens = nn.Embedding(len(tgt_dict), args.decoder_embed_dim, tgt_dict.pad())

    # 編碼器與解碼器
    # TODO: 替換成 TransformerEncoder 和 TransformerDecoder
    encoder = TransformerEncoder(args, src_dict, encoder_embed_tokens)
    decoder = TransformerDecoder(args, tgt_dict, decoder_embed_tokens)
    # encoder = RNNEncoder(args, src_dict, encoder_embed_tokens)
    # decoder = RNNDecoder(args, tgt_dict, decoder_embed_tokens)

    # 序列到序列模型
    model = Seq2Seq(args, encoder, decoder)

    # 序列到序列模型的初始化模型参数
    def init_params(module):
        from fairseq.modules import MultiheadAttention
        if isinstance(module, nn.Linear): # 对于 nn.Linear 类型的参数
            module.weight.data.normal_(mean=0.0, std=0.02) # 使用正态分布随机初始化权重，并设置mean和std参数。
            if module.bias is not None:
                module.bias.data.zero_() # 偏置初始化为 0
        if isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_() # 指定填充（padding）的位置在词汇表中的索引
        if isinstance(module, MultiheadAttention): # 对注意力机制的 q(key), k(query), v(value) 投影矩阵分别使用正态分布随机初始化
            module.q_proj.weight.data.normal_(mean=0.0, std=0.02)
            module.k_proj.weight.data.normal_(mean=0.0, std=0.02)
            module.v_proj.weight.data.normal_(mean=0.0, std=0.02)
        if isinstance(module, nn.RNNBase):
            for name, param in module.named_parameters(): # 该循环遍历其所有参数
                if "weight" in name or "bias" in name:
                    param.data.uniform_(-0.1, 0.1) # 使用均匀分布（U(-0.1, 0.1)）随机初始化该参数的值

    # 初始化模型
    model.apply(init_params)
    return model

In [ ]:
#架构相关配置

arch_args = Namespace(
    encoder_embed_dim=256,
    encoder_ffn_embed_dim=512,
    encoder_layers=6,
    decoder_embed_dim=256,
    decoder_ffn_embed_dim=1024,
    decoder_layers=6,
    share_decoder_input_output_embed=True,
    dropout=0.3, # 衰减？0.1好过0.3
)

# 可以魔改的参数？？
def add_transformer_args(args):
    args.encoder_attention_heads=4 # 多头注意力头数（即每个注意力向量拆分出的子向量数）#开8！
    args.encoder_normalize_before=True # 进行layer normalization，可以避免梯度消失问题

    args.decoder_attention_heads=4
    args.decoder_normalize_before=True

    args.activation_fn="relu" # 激活函数
    args.max_source_positions=1024 # 源语言句子的最大长度
    args.max_target_positions=1024

    # 補上我們沒有設定的Transformer預設參數
    from fairseq.models.transformer import base_architecture
    base_architecture(arch_args)

add_transformer_args(arch_args)

# 更新wandb的配置，以便在记录实验期间使用。这样，可以保存并跟踪实验中使用的所有参数和超参数
if config.use_wandb:
    wandb.config.update(vars(arch_args))


model = build_model(arch_args, task)
logger.info(model) # 用于记录信息和调试消息，将模型打印输出到log中，以方便调试和查看模型结构的展示

## Optimization 最佳化

### Loss: Label Smoothing Regularization

* 让模型学习输出不集中的分布，防止模型过度自信
* 有時候Ground Truth並非唯一答案，所以在算loss時，我們會保留一部分几率給正確答案以外的label
* 可以有效防止过拟合 Overfitting

code [source](https://fairseq.readthedocs.io/en/latest/_modules/fairseq/criterions/label_smoothed_cross_entropy.html)

### 标签平滑（Label Smoothing）

标签平滑的目的是不让模型过度依赖于某些类别，减缓过拟合风险。标签平滑过程对真实的标签进行编码，即对原有的label做如下转化：

* 对于正确的标签一部分概率分配给正确的标签；

* 对于非正确标签，将概率分配为相同值。

其中“一部分概率”由超参数 $\epsilon$ 控制，因此 $(1-\epsilon)\times100%$ 的权重分配给真实的标签。这部分权重被分散到其他标签中。每个标签都得到 $\epsilon/(n-1)$ 的概率值。例如，当 $n=10$ 时，正确标签有 $0.9$的概率，并将剩余 $0.1$ 的概率均匀分配给其他9个非正确标签，每个标签的值为 $ 0.1/9=0.011$。

In [ ]:
#自定义损失函数
class LabelSmoothedCrossEntropyCriterion(nn.Module):
    def __init__(self, smoothing, ignore_index=None, reduce=True):
         # 实现了标签平滑的交叉熵损失函数。标签平滑的目的是避免模型对于训练数据中的异常标签过于敏感
        super().__init__()
        self.smoothing = smoothing # 标签平滑的值，一般取0.1
        self.ignore_index = ignore_index # 需要忽略的标签，默认为None；
        self.reduce = reduce # 是否对损失进行求和操作，默认为True

    def forward(self, lprobs, target):
        # lprobs：预测的log softmax概率分布，形状为(batch_size, seq_length, num_classes)
        # target：目标标签，形状为(batch_size, seq_length)

        # 首先，如果目标标签的维度比预测的概率分布要低一维，则对目标标签的维度进行升维操作
        if target.dim() == lprobs.dim() - 1:
            target = target.unsqueeze(-1)

        # nll: 负对数似然，one-hot的交叉shang. following line is same as F.nll_loss
        nll_loss = -lprobs.gather(dim=-1, index=target)
        # 保留是其他label的可能性，因此算CP时.把负对数加起来
        # 將一部分正確答案的機率分配給其他label 所以當計算cross-entropy時等於把所有label的log prob加起來
        # 将预测的log softmax概率分布lprobs在最后一维上求和得到每个标签的log概率和
        smooth_loss = -lprobs.sum(dim=-1, keepdim=True)
        if self.ignore_index is not None:
            pad_mask = target.eq(self.ignore_index) #还有要masked的地方
            nll_loss.masked_fill_(pad_mask, 0.0)
            smooth_loss.masked_fill_(pad_mask, 0.0)
        else:
            nll_loss = nll_loss.squeeze(-1) #不用mask只能，直接求和
            smooth_loss = smooth_loss.squeeze(-1)
        if self.reduce: # 对损失进行求和操作
            nll_loss = nll_loss.sum() #这要sum?
            smooth_loss = smooth_loss.sum()

        # 計算cross-entropy時 加入分配給其他label的loss
        # 对交叉熵损失和平滑损失进行加权求和，得到最终的损失值。
        # 其中，平滑损失的权重为eps_i，交叉熵损失的权重为(1 - smoothing)
        eps_i = self.smoothing / lprobs.size(-1)
        loss = (1.0 - self.smoothing) * nll_loss + eps_i * smooth_loss
        return loss


# 损失函数实例化
# 平滑因子=0.1 ， target_dict就忽略？
criterion = LabelSmoothedCrossEntropyCriterion(
    smoothing=0.1,
    ignore_index=task.target_dictionary.pad(),
)

In [ ]:
#优化器：Adam+学习率计划
#通过自定义优化器实现

def get_rate(d_model, step_num, warmup_step):
    # TODO: Change lr from constant to the equation shown above
    lr = (d_model ** (-0.5)) * min(step_num ** (-0.5), step_num * warmup_step ** (-1.5))
    return lr

class NoamOpt: #自定义优化器
    def __init__(self, model_size, factor, warmup, optimizer): #信息全盘继承就完事了
        self.optimizer = optimizer #底层优化器
        self._step = 0 #初始化步数
        self.warmup = warmup #warmup步数
        self.factor = factor #设定学习率
        self.model_size = model_size #就model_size
        self._rate = 0 #初始化总体学习率

    @property #申明只读，不可修改(就const呗)

    def param_groups(self): #读取底层优化器的参数表
        return self.optimizer.param_groups

    def multiply_grads(self, c): # 将梯度乘上常数c
        # 在计算完损失函数的梯度后，将相关参数的梯度条目用常数标量c缩放。
        # 这个常数标量可以用来进行梯度裁剪操作，以避免梯度爆炸的问题
        for group in self.param_groups: #扫所有参数表（每对value-target都有个数据表）
            for p in group['params']: #读每个参数表中的params
                if p.grad is not None:
                    p.grad.data.mul_(c) #梯度存在就直接乘c

    def step(self): #更新参数与学习率
        self._step += 1 #step++
        rate = self.rate() #获取新的学习率
        for p in self.param_groups:
            p['lr'] = rate #设定每个参数表的学习率
        self._rate = rate #总体学习率
        self.optimizer.step() #底层优化器开冲

    def rate(self, step = None): #当前学习率
        if step is None:
            step = self._step
        return 0 if not step else self.factor * get_rate(self.model_size, step, self.warmup)


In [ ]:

#实例化优化器
optimizer = NoamOpt(
    model_size=arch_args.encoder_embed_dim,
    factor=config.lr_factor,
    warmup=config.lr_warmup,
    optimizer=torch.optim.AdamW(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.0001))

#学习率计划可视化
plt.plot(np.arange(1, 100000), [optimizer.rate(i) for i in range(1, 100000)])
plt.legend([f"{optimizer.model_size}:{optimizer.warmup}"])
None

訓練步驟

### 训练

#### 混合精度训练

1. 概述
  * 混合精度训练是一种**加速深度神经网络训练**的技术
  * 通过将模型参数使用较低精度的数值类型存储和更新，从而减少存储和计算的开销，提高训练效率。
  * 一般来说，使用低精度的数值类型可能会导致精度下降、梯度消失或爆炸等问题，为了解决这些问题，混合精度训练还需要使用梯度缩放、梯度转换等技术。
2. 底层
  * 在混合精度训练中，通常将模型参数划分为两类：主精度（master precision）和辅助精度（auxiliary precision）。
  * 主精度通常是32位浮点数，用于存储和更新模型参数，而辅助精度则是16位或8位浮点数，用于存储梯度和临时变量。在每次更新模型参数前，可以将梯度从辅助精度转换为主精度，计算完后再转换回来。

在 PyTorch 中，可以使用 torch.cuda.amp 模块实现自动混合精度训练，简化了混合精度训练的代码实现。

In [ ]:
# -----训练过程------
from fairseq.data import iterators
from torch.cuda.amp import GradScaler, autocast


def train_one_epoch(epoch_itr, model, task, criterion, optimizer, accum_steps=1):
    # 读下一个epoch
    itr = epoch_itr.next_epoch_itr(shuffle=True)
    # 梯度积累:更新每个accum_steps samples
    itr = iterators.GroupedIterator(itr, accum_steps)

    stats = {"loss": []}
    scaler = GradScaler() # 自动混合精度 automatic mixed precision (amp)

    model.train() #训练模式
    progress=tqdm.tqdm(itr, desc=f"train epoch {epoch_itr.epoch}", leave=False) #把扫epoch的进度可视化
    for samples in progress:
        model.zero_grad() #把梯度清零
        accum_loss = 0
        sample_size = 0
        #enumerate()函数用于将一个可遍历的数据对象（如列表、元组或字符串）组合为一个索引序列，
        #同时列出数据和数据下标，一般用在for循环当中。
        for i, sample in enumerate(samples):
            if i == 1:
                torch.cuda.empty_cache() #清空cuda缓存

            sample = utils.move_to_cuda(sample, device=device) #sample=sample.to(device)?
            target = sample["target"] #读target
            sample_size_i = sample["ntokens"] #读出单个size
            sample_size += sample_size_i #求sample总size?


            with autocast(): # 混合精度训练
                net_output = model.forward(**sample["net_input"]) #求输出(概率)
                lprobs = F.log_softmax(net_output[0], -1) #softmax，成句子
                loss = criterion(lprobs.view(-1, lprobs.size(-1)), target.view(-1)) #除了最后一维其他拉直，

                # 求loss之和
                accum_loss += loss.item()
                # 反向传播求梯度
                scaler.scale(loss).backward() #因为梯度没有清零所以累加在这里了！

        #将优化器中的梯度缩放因子还原为1
        scaler.unscale_(optimizer)
        # 除以sample_size，求得平均梯度
        optimizer.multiply_grads(1 / (sample_size or 1.0))
        #计算梯度的范数，并将其裁剪到指定的最大范数/梯度裁剪，防止爆炸
        gnorm = nn.utils.clip_grad_norm_(model.parameters(), config.clip_norm)

        scaler.step(optimizer) #更新模型的权重
        scaler.update() #更新参数

        # logging
        loss_print = accum_loss/sample_size #求得单个batch平均loss
        stats["loss"].append(loss_print)
        progress.set_postfix(loss=loss_print) #在进度条中显示损失值
        if config.use_wandb: #将指标记录
            wandb.log({
                "train/loss": loss_print,
                "train/grad_norm": gnorm.item(),
                "train/lr": optimizer.rate(),
                "train/sample_size": sample_size,
            })

    loss_print = np.mean(stats["loss"]) #求得epoch的平均loss
    logger.info(f"training loss: {loss_print:.4f}")

    return stats

In [ ]:
# 验证与推理

# fairseq 的 beam search generator对一个样本进行翻译，并获得翻译结果
# 給定模型和輸入序列，用 beam search 生成翻譯結果
sequence_generator = task.build_generator([model], config)

def decode(toks, dictionary):
    # convert from Tensor to human readable sentence
    s = dictionary.string(
        toks.int().cpu(),
        config.post_process,
    )
    return s if s else ""

def inference_step(sample, model):
    gen_out = sequence_generator.generate([model], sample)  # 对输入样本进行翻译，其中运用束搜索，包含了多个翻译假设和对应的分数
    srcs = [] # 输入的 token 源语言序列
    hyps = []  # 生成的 token 目标语言序列
    refs = [] # 目标 token 参考答案
    for i in range(len(gen_out)):
        # 對於每個 sample, 收集輸入，輸出和參考答，分别解码成字符串，稍後計算 BLEU
        srcs.append(decode(
            utils.strip_pad(sample["net_input"]["src_tokens"][i], task.source_dictionary.pad()),
            task.source_dictionary,
        ))
        hyps.append(decode(
            gen_out[i][0]["tokens"], # 0 代表取出 beam 內分數第一的輸出結果
            task.target_dictionary,
        ))
        refs.append(decode(
            utils.strip_pad(sample["target"][i], task.target_dictionary.pad()),
            task.target_dictionary,
        ))
    return srcs, hyps, refs

import shutil
import sacrebleu
# 实现了对模型进行验证，使用验证集对模型进行评估，并计算 BLEU 分数
def validate(model, task, criterion, log_to_wandb=True):
    logger.info('begin validation')
    itr = load_data_iterator(task, "valid", 1, config.max_tokens, config.num_workers).next_epoch_itr(shuffle=False)

    stats = {"loss":[], "bleu": 0, "srcs":[], "hyps":[], "refs":[]}
    srcs = []
    hyps = []
    refs = []

    model.eval()
    progress = tqdm.tqdm(itr, desc=f"validation", leave=False)
    with torch.no_grad():
        for i, sample in enumerate(progress):
            # validation loss
            sample = utils.move_to_cuda(sample, device=device)
            net_output = model.forward(**sample["net_input"])

            lprobs = F.log_softmax(net_output[0], -1)
            target = sample["target"]
            sample_size = sample["ntokens"]
            loss = criterion(lprobs.view(-1, lprobs.size(-1)), target.view(-1)) / sample_size
            progress.set_postfix(valid_loss=loss.item())
            stats["loss"].append(loss)

            # do inference
            s, h, r = inference_step(sample, model)
            srcs.extend(s)
            hyps.extend(h)
            refs.extend(r)

    tok = 'zh' if task.cfg.target_lang == 'zh' else '13a'
    stats["loss"] = torch.stack(stats["loss"]).mean().item()
    stats["bleu"] = sacrebleu.corpus_bleu(hyps, [refs], tokenize=tok) # 計算BLEU score
    stats["srcs"] = srcs
    stats["hyps"] = hyps
    stats["refs"] = refs

    if config.use_wandb and log_to_wandb:
        wandb.log({
            "valid/loss": stats["loss"],
            "valid/bleu": stats["bleu"].score,
        }, commit=False)

    showid = np.random.randint(len(hyps))
    logger.info("example source: " + srcs[showid])
    logger.info("example hypothesis: " + hyps[showid])
    logger.info("example reference: " + refs[showid])

    # show bleu results
    logger.info(f"validation loss:\t{stats['loss']:.4f}")
    logger.info(stats["bleu"].format())
    return stats

 儲存及載入模型參數

In [ ]:
# 验证并保存模型
def validate_and_save(model, task, criterion, optimizer, epoch, save=True):
    stats = validate(model, task, criterion) #把验证结果拉下来
    bleu = stats['bleu']
    loss = stats['loss']
    if save:
        # 保存模型地址
        savedir = Path(config.savedir).absolute()
        savedir.mkdir(parents=True, exist_ok=True)

        # 要保存的信息
        check = {
            "model": model.state_dict(),
            "stats": {"bleu": bleu.score, "loss": loss},
            "optim": {"step": optimizer._step}
        }

        #保存当前模型
        torch.save(check, savedir/f"checkpoint{epoch}.pt")
        shutil.copy(savedir/f"checkpoint{epoch}.pt", savedir/f"checkpoint_last.pt")
        logger.info(f"saved epoch checkpoint: {savedir}/checkpoint{epoch}.pt")

        # 保存机器翻译结果
        with open(savedir/f"samples{epoch}.{config.source_lang}-{config.target_lang}.txt", "w") as f:
            for s, h in zip(stats["srcs"], stats["hyps"]):
                f.write(f"{s}\t{h}\n")

        # 保存验证集上bleu最好的模型
        if getattr(validate_and_save, "best_bleu", 0) < bleu.score:
            validate_and_save.best_bleu = bleu.score
            torch.save(check, savedir/f"checkpoint_best.pt")

        # 保存的模型太多了就把之前的删掉
        del_file = savedir / f"checkpoint{epoch - config.keep_last_epochs}.pt"
        if del_file.exists():
            del_file.unlink()

    return stats

#加载模型
def try_load_checkpoint(model, optimizer=None, name=None):
    name = name if name else "checkpoint_last.pt" #要加载的模型名
    checkpath = Path(config.savedir)/name #全路径
    if checkpath.exists(): #如果模型存在就load
        check = torch.load(checkpath)
        model.load_state_dict(check["model"]) #模型参数
        stats = check["stats"] #blue 与 loss
        step = "unknown"
        if optimizer != None:
            optimizer._step = step = check["optim"]["step"] #已经训练好的步数
        logger.info(f"loaded checkpoint {checkpath}: step={step} loss={stats['loss']} bleu={stats['bleu']}")
    else:
        logger.info(f"no checkpoints found at {checkpath}!") #找不到就哭呗

## Main

In [ ]:
# 模型与损失函数实例化
model = model.to(device=device) #模型
criterion = criterion.to(device=device) #损失函数（用LSCE啦）

In [ ]:
# 记录训练信息
# 当前任务、编码器、解码器、损失函数和优化器的类型，以及模型的参数数量和训练配置信息
logger.info("task: {}".format(task.__class__.__name__))
logger.info("encoder: {}".format(model.encoder.__class__.__name__))
logger.info("decoder: {}".format(model.decoder.__class__.__name__))
logger.info("criterion: {}".format(criterion.__class__.__name__))
logger.info("optimizer: {}".format(optimizer.__class__.__name__))
logger.info(
    "num. model params: {:,} (num. trained: {:,})".format(
        sum(p.numel() for p in model.parameters()),
        sum(p.numel() for p in model.parameters() if p.requires_grad),
    )
)
logger.info(f"max tokens per batch = {config.max_tokens}, accumulate steps = {config.accum_steps}")


In [ ]:
# 加载训练数据迭代器，从指定的start_epoch开始训练
epoch_itr = load_data_iterator(task, "train", config.start_epoch, config.max_tokens, config.num_workers)
try_load_checkpoint(model, optimizer, name=config.resume)
while epoch_itr.next_epoch_idx <= config.max_epoch:
    #训练
    train_one_epoch(epoch_itr, model, task, criterion, optimizer, config.accum_steps)
    #验证
    stats = validate_and_save(model, task, criterion, optimizer, epoch=epoch_itr.epoch)
    #记录一下epoch结束
    logger.info("end of epoch {}".format(epoch_itr.epoch))
    #去下一个epoch
    epoch_itr = load_data_iterator(task, "train", epoch_itr.next_epoch_idx, config.max_tokens, config.num_workers)

## Submission

这段代码使用了fairseq库中的average_checkpoints.py脚本，用于将多个训练过程中的checkpoint模型进行平均，得到一个新的平均模型，其效果类似于ensemble集成方法。具体解释如下：

* --inputs {checkdir}：表示输入的checkpoint模型所在的目录路径。
* --num-epoch-checkpoints 5：表示选择最近的5个epoch的checkpoint进行平均处理。
* --output {checkdir}/avg_last_5_checkpoint.pt：表示输出平均后的模型的保存路径和文件名。

在训练过程中，使用一些技巧来提升模型性能，其中的一种方法就是使用ensemble集成技术，将多个模型的预测结果进行综合，得到更好的预测结果。而这段代码中的平均checkpoint方法能够让我们在训练过程中就使用ensemble技术，将多个模型的权重进行平均，得到一个新的平均模型，具有比单个模型更强的泛化能力和稳定性。

In [ ]:
# 把几个checkpoint取平均 ensemble
checkdir=config.savedir
!python ./fairseq/scripts/average_checkpoints.py \
--inputs {checkdir} \
--num-epoch-checkpoints 5 \
--output {checkdir}/avg_last_5_checkpoint.pt

## Confirm model weights used to generate submission

这段代码中使用了三个checkpoint模型来对模型进行验证：

* checkpoint_last.pt：选择最近一个epoch的checkpoint模型进行验证，即使用最新训练的模型。
* checkpoint_best.pt：选择在验证集上表现最好的checkpoint模型进行验证，即使用最有优的模型。
* avg_last_5_checkpoint.pt：将最近五个epoch的checkpoint模型进行平均，得到平均后的模型进行验证。

代码中使用了try_load_checkpoint函数，该函数用于加载指定名称的checkpoint模型，如果成功加载，则返回True，否则返回False。如果加载成功，则会将模型的参数更新为checkpoint中的参数。

validate函数用于在给定数据集上对模型进行验证，其中使用了task、criterion等参数，log_to_wandb参数表示是否将结果记录到wandb中。在这里，我们使用上述三个checkpoint模型对模型进行验证，并输出验证结果。

In [ ]:
# checkpoint_last.pt : latest epoch
# checkpoint_best.pt : highest validation bleu
# avg_last_5_checkpoint.pt:　the average of last 5 epochs
try_load_checkpoint(model, name="avg_last_5_checkpoint.pt")
avg = validate(model, task, criterion, log_to_wandb=False)

try_load_checkpoint(model, name="checkpoint_best.pt")
best = validate(model, task, criterion, log_to_wandb=False)

try_load_checkpoint(model, name="checkpoint_last.pt")
last = validate(model, task, criterion, log_to_wandb=False)
print("avg: ",avg['bleu'])
print("best: ", best['bleu'])
print("last: ", last['bleu'])
None

# 预测

In [ ]:

def generate_prediction(model, task, split="test", outfile="./prediction.txt"):
    #加载数据组与数据个体
    task.load_dataset(split=split, epoch=1)
    itr = load_data_iterator(task, split, 1, config.max_tokens, config.num_workers).next_epoch_itr(shuffle=False)

    idxs = []
    hyps = []

    model.eval()
    progress = tqdm.tqdm(itr, desc=f"prediction")
    with torch.no_grad():
        for i, sample in enumerate(progress):
            sample = utils.move_to_cuda(sample, device=device)
            # 翻译
            s, h, r = inference_step(sample, model)
            # 记录翻译结果
            hyps.extend(h)
            idxs.extend(list(sample['id']))

    # 按ID把翻译结果排序
    hyps = [x for _,x in sorted(zip(idxs,hyps))]
    #把结果写入文件
    with open(outfile, "w") as f:
        for h in hyps:
            f.write(h+"\n")



In [ ]:
generate_prediction(model, task)

In [ ]:
raise